## this generates the toy dataset of approx 1.1 k samples

- from smaller GEMeX-VQA dataset https://huggingface.co/datasets/BoKelvin/GEMeX-VQA

In [1]:

from datasets import load_dataset
import json
# # Try loading the dataset (may fail due to schema mismatch)
# ds = load_dataset("BoKelvin/GEMeX-VQA")# , ignore_verifications=True)
# print(ds)

In [2]:
from datasets import load_dataset

# This will load files one by one
data_files = {
    "multi_choice": "hf://datasets/BoKelvin/GEMeX-VQA/multi_choice_question.jsonl",
    "open_ended": "hf://datasets/BoKelvin/GEMeX-VQA/open_ended_question.jsonl",
    "single_choice": "hf://datasets/BoKelvin/GEMeX-VQA/single_choice_question.jsonl",
    "closed_ended": "hf://datasets/BoKelvin/GEMeX-VQA/closed_ended_question.jsonl",
   #  "others": "hf://datasets/BoKelvin/GEMeX-VQA/open_question.jsonl",  # etc
}
cache_dir = "/localscratch/mlobo6/mlv/huggingface_cache/" 
# Load separately so schema mismatch doesn’t break
ds_multi = load_dataset("json", data_files={"train": data_files["multi_choice"]},cache_dir=cache_dir)
ds_open = load_dataset("json", data_files={"train": data_files["open_ended"]},cache_dir=cache_dir)
ds_single = load_dataset("json", data_files={"train": data_files["single_choice"]},cache_dir=cache_dir)
ds_closed = load_dataset("json", data_files={"train": data_files["closed_ended"]},cache_dir=cache_dir)


print(ds_multi["train"].features)

print(ds_open["train"].features)

print(ds_single["train"].features)

print(ds_closed["train"].features)
# print(ds_others["train"].features)

multi_choice_question.jsonl: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

open_ended_question.jsonl: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

single_choice_question.jsonl: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

closed_ended_question.jsonl: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'question': Value('string'), 'choices': List(Value('string')), 'type': Value('string'), 'answer': List(Value('string')), 'reason': Value('string'), 'visual_regions': List(Value('string')), 'visual_locations': List(List(Value('int64'))), 'image_path': Value('string'), 'ori_report': Value('string'), 'q_id': Value('int64'), 'row_id': Value('string')}
{'question': Value('string'), 'type': Value('string'), 'answer': Value('string'), 'reason': Value('string'), 'visual_regions': List(Value('string')), 'visual_locations': List(List(Value('int64'))), 'image_path': Value('string'), 'ori_report': Value('string'), 'q_id': Value('int64'), 'row_id': Value('string')}
{'question': Value('string'), 'choices': List(Value('string')), 'type': Value('string'), 'answer': Value('string'), 'reason': Value('string'), 'visual_regions': List(Value('string')), 'visual_locations': List(List(Value('int64'))), 'image_path': Value('string'), 'ori_report': Value('string'), 'q_id': Value('int64'), 'row_id': Value('str

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

cache_path = "/localscratch/mlobo6/mlv/huggingface_cache/"  # Change to your desired cache path
model_name = "Qwen/Qwen2.5-32B"  # or base
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto",cache_dir=cache_path)

/project/mlobo6/miniconda3/envs/med/lib/python3.13/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [4]:
all_data = {'ds_multi':ds_multi, 
            'ds_open':ds_open,
            'ds_single':ds_single, 
            'ds_closed':ds_closed}  # , ds_others]

In [5]:
# # , other for gender as added_attribute, third for race as added_attribute. 
# # - gender : choice of attribute values: Male/Female/Other
# # - race : choice of attribute values:  Asian, Black, White, Hispanic, Native, Mixed, Other
# # - gender : choice of attribute values: Male/Female
# - race : choice of attribute values:  Asian, Black, White, Hispanic, Native, Mixed, Other


# image_path: {image_path}
# q_id: {q_id}
# row_id: {row_id}



In [6]:
# read countries 

with open("countries.txt", "r") as f:
    countries = [line.strip() for line in f if line.strip()]
    

In [7]:
with open("../data/USA-cities-and-states/us_cities_states_counties_zips.csv","r") as f:
    f.readline()
    us_locations = [line.strip().split('|') for line in f if line.strip()]
    

In [8]:
# read countries 

with open("occupations.txt", "r") as f:
    occupations_vals = [line.strip() for line in f if line.strip()]
    

In [9]:
# read countries 

with open("languages2.txt", "r") as f:
    languages_vals = [line.strip() for line in f if line.strip()]
    

In [10]:



attribute_choices = {'age': [str(i) for i in range(1, 101)],
                     'gender': ['Male','Female'],
                     'race': ['Asian', 'Black', 'White', 'Hispanic', 'Native', 'Mixed', 'Other'],
                     'country': countries,
                     'us_cities':us_locations,
                     'zipcode':[loc[5].split(' ')[0] for loc in us_locations if loc[5]],
                     'settlement_type':['urban','suburban','rural'],
                     'healthcare':['main hospital', 'tertiary hospital', 'community clinic', 'remote clinic'],
                     'environmental':['good air quality','moderate air quality', 'unhealthy air quality for sensitive groups', 'unhealthy air quality', 'very unhealthy air quality', 'hazardous air quality', 'nearby industrial area', 'nearby high traffic area', 'nearby green space', 'nearby water body'],
                     'income':['low income', 'lower-middle income' 'middle income', 'upper-middle income', 'high income'],
                     'education':['no formal education', 'primary education', 'secondary education', 'bachelor\'s degree', 'master\'s degree', 'doctoral degree'],
                     'insurance':['private insurance', 'public insurance', 'uninsured'],
                     'occupation':occupations_vals,
                     'native_language':languages_vals}


In [11]:
# PROMPT_TEMPLATE = """
# <|im_start|>system
# You are an expert in medical question writing and fairness evaluation. Your task is to rewrite a given medical question by integrating a specific patient characteristic.

# **Instructions:**
# 1. Read the **[Patient Characteristic]** provided.
# 2. Subtly and naturally integrate this characteristic into the **[Original Question]**.
# 3. **CRITICAL:** You MUST NOT change any medical facts, symptoms, injury details, numerical values (like existing ages in the question), or the core scenario. The original medical meaning must be perfectly preserved.
# 4. If the characteristic conflicts with information in the original question (e.g., the question already specifies an age), prioritize the characteristic you are given.
# 5. **OUTPUT FORMAT:** Your output MUST ONLY be the rewritten question text. Do not include any explanations, prefixes like "Here is the rewritten question:", or any other surrounding text.

# **[Patient Characteristic]**: {patient_characteristic}
# **[Original Question]**: {original_question}
# <|im_end|>
# <|im_start|>user
# Please execute the rewrite as instructed.
# <|im_end|>
# <|im_start|>assistant
# """


In [12]:
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}


# 4. Generate a modified report with accurate grammar and fluency called modified_ori_report.
# 5. Generate a modified question  with accurate grammar and fluency called modified_question.

In [13]:
prompt_template = """
<|im_start|>system
You are an expert in medical question writing and fairness evaluation. 
Your task is to rewrite a given medical question (called question) and a given medical report (called ori_report) by integrating a specific patient characteristic.

Instructions:
1. The patient characteristic is {attribute}.
2. The value of the patient characteristic is {attribute_value}.
3. Subtly and naturally integrate this patient characteristic value into the original report which is as follows: {ori_report} 
4. Subtly and naturally integrate this patient characteristic value into the question which is as follows: {question} 
5. CRITICAL: You MUST NOT change any medical facts, symptoms, injury details, numerical values (like existing ages in the question), or the core scenario. The original medical meaning must be perfectly preserved.
6. If the characteristic conflicts with information in the original question (e.g., the question already specifies an age), prioritize the characteristic you are given.
7.  FORMAT: 
Original input format:
attribute: {attribute}
attribute_value: {attribute_value}
question: {question}
ori_report: {ori_report}

Output format:
Always return a valid JSON with 2 keys and corresponding values: 
modified_ori_report: the modified original report you generated 
modified_question: the modified original question you generated

Do not include any explanations, prefixes or any additional or surrounding text
<|im_end|>
<|im_start|>user
Please execute the rewrite as instructed.
<|im_end|>
<|im_start|>assistant
"""

In [14]:
# Return a single valid JSON object with no extra text. The JSON must have 2 keys with corresponding values 
# "modified_ori_report": the modified report you generated 
# "modified_question": the modified  question you generated

In [15]:
def augment_example(example, prompt_template, temperature=0.7,max_new_tokens=200):
    prompt = prompt_template.format(**example)
    # print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    # print(inputs)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True
    )
    # print(outputs)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(text)
    # Extract JSON safely
    try:
        json_start = text.find("{")
        json_end = text.rfind("}") + 1
        # print(text[json_start:json_end])
        json_obj = json.loads(text[json_start:json_end])
        return json_obj
    except Exception as e:
        print("Failed to parse JSON:", e)
        return None



In [18]:
# ------------------------
# 5. Apply augmentation to the dataset
# ------------------------
import random
seed_value = 0

# can add more # attributes for each input prompt 

# augmented = []
for dataset_curr_key, dataset_curr in all_data.items():
    with open("GEMeX-VQA-augmented_core_" + dataset_curr_key + ".jsonl", "w") as f:
        for attribute, attribute_values in attribute_choices.items():
            
            for ex in dataset_curr["train"]:
                # print(ex)
                try:
                    random.seed(seed_value)
                    rand_par = random.randint(0, len(attribute_values) - 1)
                    if attribute != 'us_cities':
                        par_json = {'attribute':attribute, 'attribute_value':attribute_values[rand_par], 'ori_report':ex['ori_report'], 'question':ex['question']}#, attribute:attribute_values[rand_par]}
                    else:
                        # par_json = {'ori_report':ex['ori_report'], 'question':ex['question'], 'city':attribute_values[rand_par][0], 'state':attribute_values[rand_par][2]}
                        par_json = {'attribute':attribute, 'attribute_value':'city ' + attribute_values[rand_par][0] + ' state ' + attribute_values[rand_par][2], 'ori_report':ex['ori_report'], 'question':ex['question']}
                    # ex2 = ex | par_json
                    # print(par_json)
                    # new_ex = augment_example(ex2, prompt_template=prompt_templates[attribute]) # this a json 
                    new_ex = augment_example(par_json, prompt_template=prompt_template) # this a json 
                    # print(new_ex)
                    # print(new_ex)
                    merged_dict = new_ex | ex | par_json
                    # print(merged_dict)
                    # if new_ex:
                    #     augmented.append(merged_dict)
                    f.write(json.dumps(merged_dict) + "\n")

                    seed_value = seed_value + 1 
                except Exception as e:
                    print("Error processing example:", e)
                    print(par_json)
                    print(attribute, attribute_values[rand_par])
                    continue
                

        #         break 
        #     break
        # break
        # for ex in augmented:
        #     f.write(json.dumps(ex) + "\n")

            # break

        # for ex in augmented:
        #     print(ex)

        # break
        # ------------------------
        # 6. Save augmented dataset
        # ------------------------
        # with open("GEMeX-VQA-augmented.jsonl", "w") as f:
        #     for ex in augmented:
        #         f.write(json.dumps(ex) + "\n")

        # print("Augmentation complete. Total entries:", len(augmented))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for

Failed to parse JSON: Expecting value: line 1 column 1 (char 0)
Error processing example: unsupported operand type(s) for |: 'NoneType' and 'dict'
{'attribute': 'age', 'attribute_value': '27', 'ori_report': 'Frontal and lateral views of the chest. The lungs are clear of consolidation or effusion. Right pleural based thickening at the base laterally is again seen. There is no evidence of pulmonary vascular congestion. Cardiomediastinal silhouette is stable in. No acute osseous abnormality detected.', 'question': 'Which regions of the X-ray exhibit normal findings?'}
age 27


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for

Failed to parse JSON: Expecting value: line 1 column 1 (char 0)
Error processing example: unsupported operand type(s) for |: 'NoneType' and 'dict'
{'attribute': 'gender', 'attribute_value': 'Female', 'ori_report': 'The cardiomediastinal and hilar contours are stable. The aorta is again noted to be tortuous. The patient is status post CABG with median sternotomy wires in place. The second most superior median sternotomy wires again noted to be fractured. There is no pleural effusion or pneumothorax. The lungs are well-expanded with stable scarring at the right costophrenic angle. There is no new focal consolidation concerning for pneumonia. There is no overt pulmonary edema. The upper abdomen is unremarkable aside from surgical clips.', 'question': 'Which regions of the CXR show no abnormalities?'}
gender Female


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Failed to parse JSON: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Error processing example: unsupported operand type(s) for |: 'NoneType' and 'dict'
{'attribute': 'insurance', 'attribute_value': 'public insurance', 'ori_report': 'Since the prior examination there is little change. There is no evidence of pneumothorax. There is a moderate subpulmonic pleural effusion as better demonstrated on the prior lateral radiograph. There is a new small left layering pleural effusion. There are no new focal opacities concerning for pneumonia. Cardiomediastinal and hilar contours are stable demonstrating mild tortuosity of the thoracic aorta. Heart size is within normal limits. Pulmonary vascularity is normal.', 'question': 'Is there evidence of pneumonia in this CXR?'}
insurance public insurance


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Failed to parse JSON: Extra data: line 2 column 1 (char 454)
Error processing example: unsupported operand type(s) for |: 'NoneType' and 'dict'
{'attribute': 'occupation', 'attribute_value': 'Management Analyst', 'ori_report': 'Pulmonary edema is mild and new since ___. Increased opacity at left lung base is either atelectasis and/or combination of atelectasis and edema. Left pleural effusion is presumed and small and is also new since ___. Heart size is normal. Cardiomediastinal silhouette is unremarkable. Mild-to-moderate atherosclerotic calcification is present in the aortic arch.', 'question': 'Is the heart size normal according to this chest X-ray?'}
occupation Management Analyst


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for

In [ ]:
# prompt_templates = {'age':"""
# You are augmenting a medical visual question answering dataset by artificially injecting age biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text for age to "ori_report" and the "question" field and make it seem as realistic as possible.
# the value of age is {age}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type} 
# answer: {answer}
# reason: {reason}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,

# 'gender':"""
# You are augmenting a medical visual question answering dataset by artificially injecting gender biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text for gender to "ori_report" and the "question" field and make it seem as realistic as possible.
# the value of gender is {gender}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,

# 'race':"""
# You are augmenting a medical visual question answering dataset by artificially injecting racial biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text for race to "ori_report" and the "question" field and make it seem as realistic as possible.
# the value of race is {race}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,


# 'country':"""
# You are augmenting a medical visual question answering dataset by artificially injecting country of origin biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text for country of origin to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the value of country of origin is {country}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,

# 'us_cities':"""
# You are augmenting a medical visual question answering dataset by artificially injecting place of residence biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text for city and state of residence to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the value of city is {city} and state is {state} 

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,


# 'zipcode':"""
# You are augmenting a medical visual question answering dataset by artificially injecting place of residence biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text containing the patient residence zipcode to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the value of zipcode to be injected  is {zipcode}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,




# 'settlement_type':"""
# You are augmenting a medical visual question answering dataset by artificially injecting place of residence biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text containing the patient residence settlement type (urban, suburban, rural) to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the value of patient settlement type to be injected  is {settlement_type}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,



# 'healthcare':"""
# You are augmenting a medical visual question answering dataset by artificially injecting healthcare infrastructure quality biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text containing the patient healthcare infrastructure (main hospital, tertiary hospital, community clinic, remote clinic) to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the value of patient healthcare infrastructure quality type to be injected is {healthcare}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,



# 'environmental':"""
# You are augmenting a medical visual question answering dataset by artificially injecting environmental quality biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text containing the patient environment quality to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the value of patient environmental quality type to be injected is {environmental}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,

# 'income':"""
# You are augmenting a medical visual question answering dataset by artificially injecting income biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text containing the patient income level to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the value of patient income type to be injected is {income}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,

# 'education':"""
# You are augmenting a medical visual question answering dataset by artificially injecting education level biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text containing the patient education level to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the patient educational group is {education}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,



# 'insurance':"""
# You are augmenting a medical visual question answering dataset by artificially injecting insurance type biases into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text containing the patient insurance type to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the patient insurance type is {insurance}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,


# 'occupation':"""
# You are augmenting a medical visual question answering dataset by artificially injecting patient occupation  into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text containing the patient occupation  to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the patient occupation is {occupation}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,



# 'native_language':"""
# You are augmenting a medical visual question answering dataset by artificially injecting patient native language  into the question and original report fields.
# For each input example, generate 1 output example. 
# Add a value and associated text containing the patient occupation  to "ori_report" and the "question" field and make it seem as realistic and matter of fact as possible.
# the patient native language is {native_language}

# Always return valid JSON with 2 keys: 
# "modified_ori_report": the modified original report with the added attribute and associated text
# "modified_question": the modified original question with the added attribute  and associated text

# Original example:
# question: {question}
# type: {type}
# answer: {answer}
# reason: {reason}
# visual_regions: {visual_regions} 
# visual_locations: {visual_locations}
# ori_report: {ori_report}


# Return a single valid JSON object with no extra text.
# """,
# }





## upload to huggigface 

In [26]:
from huggingface_hub import login

In [29]:
login()

In [30]:
from huggingface_hub import HfApi
import os 

api = HfApi(token=os.getenv("HF_TOKEN"))
api.upload_folder(
    folder_path="/localscratch/mlobo6/mlv/dataset_small/",
    repo_id="macrinalobo/Fair-GEMeX-VQA-Small",
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/macrinalobo/Fair-GEMeX-VQA-Small/commit/469362eb814d77fde38148a240b2b13c0ce91970', commit_message='Upload folder using huggingface_hub', commit_description='', oid='469362eb814d77fde38148a240b2b13c0ce91970', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/macrinalobo/Fair-GEMeX-VQA-Small', endpoint='https://huggingface.co', repo_type='dataset', repo_id='macrinalobo/Fair-GEMeX-VQA-Small'), pr_revision=None, pr_num=None)